In [1]:
from tqdm import tqdm
import glob
import json
import os
import re
import csv
import pandas as pd

## fine-tuning_tsv

In [2]:
def load_json_data(path):

    with open(path) as f:
        data = json.load(f)

    ids = []
    dialogues = []
    summaries = []
    topic = []
    for datum in data["data"]:
        ids.append(datum["header"]["dialogueInfo"]["dialogueID"])

        prev_speaker_id = None
        prev_line = ""
        utts = []
        for dialogue in datum["body"]["dialogue"]:
            utterance = dialogue["utterance"].strip()

            if dialogue["participantID"] == prev_speaker_id:
                prev_line += " " + utterance
            else:
                if prev_line:
                    utts.append(prev_line)
                prev_line = utterance
                prev_speaker_id = dialogue["participantID"]
        if prev_line:
            utts.append(prev_line)

        dialogues.append(utts)
        summaries.append(datum["body"].get("summary"))

    for i in range(len(data['data'])):
      topic.append(data['data'][i]['header']['dialogueInfo']['topic'])
    return ids, dialogues, summaries, topic

def data_load(filename, is_meta=False):
    ids_list, dialogues_list, summaries_list, topic_list = [], [], [], []
    dialogues_sep = []

    for file in tqdm(filename):
      ids, dialogues, summaries, topic = load_json_data(file)
      for id, text, summ, top in zip(ids, dialogues, summaries, topic):
        ids_list.append(id)
        if is_meta:
          text.insert(0,"#"+top+"#")
        dialogues_list.append(text)
        summaries_list.append(summ)
        topic_list.append(top)
    
    for text in tqdm(dialogues_list):
      dialogues_sep.append("[sep]".join(text))

    return ids_list, dialogues_sep, summaries_list


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@,-\[\]\(\)]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    
    return sentence

def data_process(data):
  # 전체 Text 데이터에 대한 전처리 (1)
  text = []

  for data_text in tqdm(data):
    text.append(preprocess_sentence(data_text))
  
  return text

In [4]:

dirname = "/content/drive/MyDrive/인공지능/아이펠톤/PoC/kt_data/Training"
filenames = os.listdir(dirname) 
train_full_filename = []

for filename in filenames:
    fn = os.path.join(dirname, filename)
    if dirname + '/.ipynb_checkpoints' != fn:
        train_full_filename.append(fn)

dirname2 = "/content/drive/MyDrive/인공지능/아이펠톤/PoC/kt_data/Validation"
filenames2 = os.listdir(dirname2) 
val_full_filename = []

for filename in filenames2:
    fn2 = os.path.join(dirname2, filename)
    if dirname + '/.ipynb_checkpoints' != fn2:
        val_full_filename.append(fn2)
     

In [5]:
train_ids, train_dialogues, train_summaries = data_load(train_full_filename, is_meta=True)
val_ids, val_dialogues, val_summaries = data_load(val_full_filename)

train_texts = data_process(train_dialogues)
val_texts = data_process(val_dialogues)

100%|██████████| 35004/35004 [00:00<00:00, 36394.92it/s]


In [6]:
train_texts[0]

'#상거래(쇼핑)#[sep]그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?[sep]우리 비행포함 15일이야??[sep]16일 아 #@이름#아 너 나한테 돈 보내주면 지금 할 수 잇옹[sep]얼마야 최종 결제액이?[sep]잠시만 인당 952,900 합쳐서 1,905,800 근데 나중에 특가 뜰 수도 있으려나..?[sep]좀 더 두고볼까? 뜨기야 뜨겠지,,[sep]웅웅 보니까 아시아나는 특가 이벤트 꽤 하는 것 같아서 일단 두고보장[sep]그래 구럼 일단 자자'

In [7]:
with open('/content/drive/MyDrive/인공지능/생성요약프로젝트/data/train.tsv', 'w', encoding='utf-8', newline='') as f:
  tw = csv.writer(f, delimiter='\t')
  tw.writerow(['dialogues', 'summaries'])

  for i in range(len(train_texts)):
    tw.writerow([train_texts[i], train_summaries[i]])

with open('/content/drive/MyDrive/인공지능/생성요약프로젝트/data/val.tsv', 'w', encoding='utf-8', newline='') as f:
  tw = csv.writer(f, delimiter='\t')
  tw.writerow(['dialogues', 'summaries'])

  for i in range(len(val_texts)):
    tw.writerow([val_texts[i], val_summaries[i]])

In [8]:
test = pd.read_csv("/content/drive/MyDrive/인공지능/생성요약프로젝트/data/train.tsv", delimiter='\t')
test

,dialogues,summaries
0,#상거래(쇼핑)#[sep]그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정...,"비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다."
1,#상거래(쇼핑)#[sep]kf마스크만 5부제 하는거지?[sep]응. 면마스크는 아무...,비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.
2,#상거래(쇼핑)#[sep]아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
3,#상거래(쇼핑)#[sep]칫솔사야하는데 쓱으로 살까?[sep]뭘 칫솔사는것까지 물어...,칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.
4,#상거래(쇼핑)#[sep]잠도안오네 얼릉 고구마츄 먹고싶단[sep]그게 그렇게 맛있...,잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.
...,...,...
279987,#행사#[sep]도착하샸나염 자리잡고 알려주이소[sep]아아 나 다이소좀 구경하느랔...,역 근처에 있는 곳이 리모델링을 하여 더 좋을 거 같아 그쪽으로 가고 있다.
279988,#행사#[sep]시간잘봐라 겁나 여러가지다[sep]예약내역 올려바바[sep]그러게[...,교통편 및 호텔 예약과 관련한 사항을 확인한다.
279989,#행사#[sep]언제 도착요정이십니까[sep]15분뒤 도착이룝 [sep]옥희여[se...,가고 있는 길을 설명해 주고 15분 뒤 도착이라고 한다.
279990,#행사#[sep]근데 현인가요제 가면 최소20시간은 줄서서 기다려야하는거아님? [s...,저녁에 하는 현인가요제는 아침부터 12시간은 줄을 서야 들어갈 수 있다.


In [9]:
test = pd.read_csv("/content/drive/MyDrive/인공지능/생성요약프로젝트/data/val.tsv", delimiter='\t')
test

,dialogues,summaries
0,웅[sep]영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소...,팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.
1,너는 잘가라....회사.... 선택 잘해..[sep]알겠어 많이 힘들구나... 나도...,이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다.
2,느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 [sep]그러면서[sep]샘...,느낌상 대통령까지는 아니고 오시면 여사님 정도 오시지 않을까라며 이에 대해 이야기하...
3,숨만수이 도 숨만쉬어도 100 이내[sep]한달안에 일 무조건 해야대[sep]아 딱...,한 달 안에 무조건 일을 시작해서 돈을 벌어야 한다.
4,목요일은 외근이구 금요일은 출장 [sep]금요일이 당진이양?[sep]아닝아닝 10일...,목요일에 외근이고 금요일에 출장인데 당진은 10일에 간다.
...,...,...
34999,방금 샐러드 정기배송 주문한 거 먹었눈데 #@시스템#사진# 왜 먹어도 배고푸야[se...,다이어트 도시락 먹다가 너무 물리고 별로라 샐러드 정기배송을 시켜 먹는데 생각보다 ...
35000,닭갈비 맛있었다 맛나맛나 그티웅??[sep] 아깐 먹기 싫다더니 [sep]역시 너의...,아깐 먹기 싫다고 했지만 닭갈비 맛있었으며 역시 너의 선택은 최고이고 볶음밥도 최고...
35001,나 핫바 먹어야지 먹을래?[sep]아니 난 안 무거[sep]그럼 나만 먹는다잉[se...,핫바를 먹으려고 하는데 어떤 소스가 맛있을지 묻고 있다.
35002,아 본죽에 신메뉴 나온 거 알아???[sep]오 신메뉴 나온 걸 왜 못봤지 ? 언니...,본죽에서 신메뉴로 6쪽 마늘 닭죽이 나왔는데 마늘이 토핑처럼 들어가 있다.


## pretrain_tsv

In [2]:
def data_mining(data):
  body_list = []
  text_list = []
  total_list = []
  with open(data) as f:
        data = json.load(f)
  for datum in data["data"]:
      body_list.append(datum['body'])  
  for text in body_list:
    text_list.append(text)
  for utt in text_list:
      utt_list = []
      for i in range(len(utt)):
          utt_list.append(utt[i]['utterance'])
      total_list.append(utt_list)
  return total_list

def data_load(filename):
    text = []

    for file in tqdm(filename):
      total_list = data_mining(file)
      for data in total_list:
        text.append("[sep]".join(data))

    return text

In [3]:
dirname = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/[라벨]한국어SNS_train"
filenames = os.listdir(dirname) 
train_full_filename = []

for filename in filenames:
    fn = os.path.join(dirname, filename)
    if dirname + '/.ipynb_checkpoints' != fn:
        train_full_filename.append(fn)

dirname2 = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/[라벨]한국어SNS_valid"
filenames2 = os.listdir(dirname2) 
val_full_filename = []

for filename in filenames2:
    fn2 = os.path.join(dirname2, filename)
    if dirname + '/.ipynb_checkpoints' != fn2:
        val_full_filename.append(fn2)

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@,-\[\]\(\)]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    
    return sentence

def data_process(data):
  # 전체 Text 데이터에 대한 전처리 (1)
  text = []

  for data_text in tqdm(data):
    text.append(preprocess_sentence(data_text))
  
  return text

In [5]:
train_dataset = data_load(train_full_filename)
train_list = data_process(train_dataset)

val_dataset = data_load(val_full_filename)
val_list = data_process(val_dataset)

100%|██████████| 200004/200004 [00:05<00:00, 37951.72it/s]


In [6]:
with open('/content/drive/MyDrive/인공지능/생성요약프로젝트/data/pretrain_train.tsv', 'w', encoding='utf-8', newline='') as f:
  tw = csv.writer(f, delimiter='\t')
  tw.writerow(['dialogues'])

  for i in range(len(train_list)):
    tw.writerow([train_list[i]])

with open('/content/drive/MyDrive/인공지능/생성요약프로젝트/data/pretrain_val.tsv', 'w', encoding='utf-8', newline='') as f:
  tw = csv.writer(f, delimiter='\t')
  tw.writerow(['dialogues'])

  for i in range(len(val_list)):
    tw.writerow([val_list[i]])

In [7]:
test = pd.read_csv("/content/drive/MyDrive/인공지능/생성요약프로젝트/data/pretrain_train.tsv", delimiter='\t')
test

,dialogues
0,두번의[sep]싣로[sep]8일만에 사망..[sep]시도했 [sep]헙[sep]헉[...
1,코로나...때문이 아닐까 추측...[sep]대눈물.....[sep]하나 정해야 하니...
2,뭐햇???[sep]오늘 나 시험 10신데 9시 35분에 일어나서 택시타고 옴[sep...
3,와 보령 2.5단계로 올린대[sep]미쳤어 다른지역은 다 내리는 마당에 여기는 올렸...
4,내가 해먹지뭐[sep]#@이름#가 해주는거 먹으랴면 10년은 기다려야하자냐[sep]...
...,...
1599987,#@시스템#사진#[sep]설마[sep]서울이자나.....[sep]서울만이기를... ...
1599988,근데 아직 두번 남아서 그냥 참게 흑흑[sep]그래 아직 두번남았으니까[sep]마자...
1599989,어제 1시반에[sep]일어나서[sep]한시간 동안 못잠[sep]나도 새벽에 한 두번...
1599990,외국인들이 우리나라 공항와서[sep]너무 좋다고 깜짝 놀라네[sep]좋은 편이지[s...


In [8]:
test = pd.read_csv("/content/drive/MyDrive/인공지능/생성요약프로젝트/data/pretrain_val.tsv", delimiter='\t')
test

,dialogues
0,보고싶더랴.. 오라고는 못하겠고[sep]아이고..담주에 가야겠네[sep]가끔 가면 ...
1,아니 나 다음주까지 완결치는게 목표인데[sep]의자 안옵니까?[sep][sep]개빡...
2,맞아 현실적인 조언이군[sep]근데 난 오히려 저런 현실적인 조언이 조아[sep]#...
3,나 엘레베이터안인데[sep]애기가 나 발로[sep]차서[sep]어엉 왜차 [sep]...
4,#@기타#데이에[sep]다들 시다 뛸 준비하고와[sep]#@이름#시켜[sep]#@이...
...,...
199999,어머니[sep]저 집 들어가면서 스타벅스와 올리브영을 들릴 건데[sep]같이 가 줄...
200000,흠 어렵네...그건[sep]자갸 나 오늘 7시쯤 끝날것 같넹[sep]괜차나..?[s...
200001,#@이름# 오면[sep]할 거 많다[sep]맞앜[sep]우선 엽떡과 노래방은[sep...
200002,다음날은 일어나서[sep]근처 또 유명빵집 가서[sep]점심을 먹고[sep]집으로 ...


In [9]:
import tarfile

filename = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/train.tsv"
compressed_filename = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/train.tar.gz"

with tarfile.open(compressed_filename, "w:gz") as tar:
    tar.add(filename)

filename = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/val.tsv"
compressed_filename = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/val.tar.gz"

with tarfile.open(compressed_filename, "w:gz") as tar:
    tar.add(filename)

filename = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/pretrain_train.tsv"
compressed_filename = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/pretrain_train.tar.gz"

with tarfile.open(compressed_filename, "w:gz") as tar:
    tar.add(filename)

filename = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/pretrain_val.tsv"
compressed_filename = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/pretrain_val.tar.gz"

with tarfile.open(compressed_filename, "w:gz") as tar:
    tar.add(filename)